## Level0 Nodes

In [72]:
import torch
import torchhd

from src.encoding.configs_and_constants import VSAModel
from src.utils.utils import cartesian_bind_tensor, TupleIndexer, flatten_counter, DataTransformer
from collections import defaultdict, Counter
## Max HRR 0.851063829787234
## Max MAP 0.6666666666666667
vsa = VSAModel.HRR
D = 96 * 96  # 96 didn't give 100%
fruits = torchhd.random(28, D, vsa=vsa.value)
veggies = torch.cat([torchhd.identity(1, D, vsa=vsa.value), torchhd.random(5, D, vsa=vsa.value)], dim=0)
node_codebook = cartesian_bind_tensor([fruits, veggies])
node_indexer = TupleIndexer([len(fruits), len(veggies)])

# Length 33
basket = [
    (0, 1),
    (0, 2),
    (0, 2),
    (0, 2),
    (0, 2),
    (0, 2),
    (0, 3),
    (2, 2),
    (2, 3),
    (0, 3),
    (2, 1),
    (0, 3),
    (0, 2),
    (0, 3),
    (0, 3),
    (0, 1),
    (2, 3),
    (0, 3),
    (0, 2),
    (0, 2),
    (0, 3),
    (0, 1),
    (0, 2),
    (0, 3),
    (0, 1),
    (0, 3),
    (0, 1),
    (0, 2),
    (0, 3),
    (1, 1),
    (2, 2),
    (0, 3),
    (5, 2),
]

print(basket)
print(len(basket))
unique_nodes = set(basket)
print(f"Unique Nodes -> {unique_nodes=}, {len(set(basket))=} ")
nodes_ground_truth = node_indexer.get_idxs(basket)
print(f"{sorted(set(nodes_ground_truth))=}")

nodes = [node_codebook[node_indexer.get_idx(t)] for t in basket]
embedding_0 = torchhd.multiset(torch.stack(nodes, dim=0))

[(0, 1), (0, 2), (0, 2), (0, 2), (0, 2), (0, 2), (0, 3), (2, 2), (2, 3), (0, 3), (2, 1), (0, 3), (0, 2), (0, 3), (0, 3), (0, 1), (2, 3), (0, 3), (0, 2), (0, 2), (0, 3), (0, 1), (0, 2), (0, 3), (0, 1), (0, 3), (0, 1), (0, 2), (0, 3), (1, 1), (2, 2), (0, 3), (5, 2)]
33
Unique Nodes -> unique_nodes={(0, 1), (2, 1), (1, 1), (0, 3), (2, 3), (0, 2), (2, 2), (5, 2)}, len(set(basket))=8 
sorted(set(nodes_ground_truth))=[1, 2, 3, 7, 13, 14, 15, 32]


In [73]:
print(sum([d for _, d in basket]))

72


### Decoding Level 0 Nodes

In [74]:
from collections import Counter

d = torchhd.dot(embedding_0, node_codebook)
if vsa == VSAModel.MAP:
    d /= D
sim_node = torch.round(d).int().clamp(min=0)
## This should print 3, 0, 1, 0, 1, 0, 3, .... so many time as there items.
## And it does, so it should work
print(sim_node)
print(sim_node.sum().item())
decoded_node_idxs = sim_node.nonzero(as_tuple=True)[0]
decoded_node_counts = sim_node[decoded_node_idxs]
# Build the counter
node_counter = Counter(dict(zip(decoded_node_idxs.tolist(), decoded_node_counts.tolist())))
print(f"{node_counter=}")
tuple_counter  = Counter(dict(zip(map(lambda x: node_indexer.get_tuple(x), decoded_node_idxs.tolist()), decoded_node_counts.tolist())))
print(f"{tuple_counter=}")
nodes_decoded_flatt = flatten_counter(node_counter)
print(f"{nodes_decoded_flatt=}")
nodes_basket_dec = list(map(lambda x: node_indexer.get_tuple(x), nodes_decoded_flatt))
print(f"{nodes_basket_dec=}")
print(f"{sorted(nodes_basket_dec) == sorted(basket)=}")

HRRTensor([ 0,  5, 10, 11,  0,  0,  0,  1,  0,  0,  0,  0,  0,  1,  2,  2,  0,
            0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,
            0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
            0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
            0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
            0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
            0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
            0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
            0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
            0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
          dtype=torch.int32)
33
node_counter=Counter({3: 11, 2: 10, 1: 5, 14: 2, 15: 2, 7: 1, 13: 1, 32: 1})
tuple_counter=Counter({(0, 3): 11, (0, 2): 10, (0, 1): 5, (2, 2): 2, (2, 3): 2, (1, 1): 1, (2, 1): 1, (5, 2)

## Level 1 - GraphHD Encoding

In [75]:
## Length 72
edge_indexes = [
    (0, 1),
    (1, 0),
    (1, 2),
    (2, 1),
    (2, 3),
    (3, 2),
    (3, 4),
    (4, 3),
    (4, 5),
    (5, 4),
    (5, 6),
    (6, 5),
    (6, 7),
    (6, 32),
    (7, 6),
    (7, 8),
    (8, 7),
    (8, 9),
    (8, 31),
    (9, 8),
    (9, 10),
    (9, 11),
    (10, 9),
    (11, 9),
    (11, 12),
    (11, 28),
    (12, 11),
    (12, 13),
    (13, 12),
    (13, 14),
    (13, 27),
    (14, 13),
    (14, 15),
    (14, 16),
    (15, 14),
    (16, 14),
    (16, 17),
    (16, 25),
    (17, 16),
    (17, 18),
    (17, 23),
    (18, 17),
    (18, 19),
    (19, 18),
    (19, 20),
    (20, 19),
    (20, 21),
    (20, 22),
    (21, 20),
    (22, 20),
    (22, 23),
    (23, 17),
    (23, 22),
    (23, 24),
    (24, 23),
    (25, 16),
    (25, 26),
    (25, 27),
    (26, 25),
    (27, 13),
    (27, 25),
    (28, 11),
    (28, 29),
    (28, 30),
    (29, 28),
    (30, 28),
    (30, 31),
    (31, 8),
    (31, 30),
    (31, 32),
    (32, 6),
    (32, 31),
]

print(f"{len(edge_indexes)=}")

edge_dict = defaultdict(list)
for src_tp, dst_tp in edge_indexes:
    edge_dict[src_tp].append(dst_tp)
print(edge_dict)
print(len(edge_dict))

## Create the memories for each node within the graph, the memory of a node is bundling of hv of all it's neighbours
memories = []
for _, dsts in edge_dict.items():
    memories.append(
        torchhd.multiset(torch.stack([node_codebook[node_indexer.get_idx(basket[d])] for d in dsts], dim=0))
    )

memory_codebook = torch.stack(memories)
assert memory_codebook.shape == (len(basket), D)


src_idxs = torch.tensor([node_indexer.get_idx(basket[s]) for s in edge_dict])

## Create GraphHD like Graph
stacked = torch.stack([node_codebook[src_idxs], memory_codebook], dim=0).transpose(0, 1)
bound = torchhd.multibind(stacked)
graph_hd_level1 = torchhd.multiset(bound)

## Easier way to perform the same
graph_hd_level1_hashmap = torchhd.hash_table(node_codebook[src_idxs], memory_codebook)
assert torch.equal(graph_hd_level1, graph_hd_level1_hashmap)

len(edge_indexes)=72
defaultdict(<class 'list'>, {0: [1], 1: [0, 2], 2: [1, 3], 3: [2, 4], 4: [3, 5], 5: [4, 6], 6: [5, 7, 32], 7: [6, 8], 8: [7, 9, 31], 9: [8, 10, 11], 10: [9], 11: [9, 12, 28], 12: [11, 13], 13: [12, 14, 27], 14: [13, 15, 16], 15: [14], 16: [14, 17, 25], 17: [16, 18, 23], 18: [17, 19], 19: [18, 20], 20: [19, 21, 22], 21: [20], 22: [20, 23], 23: [17, 22, 24], 24: [23], 25: [16, 26, 27], 26: [25], 27: [13, 25], 28: [11, 29, 30], 29: [28], 30: [28, 31], 31: [8, 30, 32], 32: [6, 31]})
33


## Explain away

In [76]:

node_counter = Counter(dict(zip(decoded_node_idxs.tolist(), decoded_node_counts.tolist())))
print(node_counter)
# print(edge_indexes)
real_edges = [(node_indexer.get_idx(basket[u]), node_indexer.get_idx(basket[v]))for u, v in edge_indexes]
print(real_edges)
# print([(node_indexer.get_idx(basket[u]), node_indexer.get_idx(basket[v]))for u, v in edge_indexes])
real_edge_counter = Counter(real_edges)
print(real_edge_counter)

nodes_decoded_flatt = sorted(list(set(nodes_decoded_flatt)))
found = set()
import itertools
edge_to_check = list(itertools.product(nodes_decoded_flatt, nodes_decoded_flatt))

# Create GraphHD like Graph
stacked = torch.stack([node_codebook[src_idxs], memory_codebook], dim=0).transpose(0, 1)
bound = torchhd.multibind(stacked)
graph_hd_level1 = torchhd.multiset(bound)

# edge_bindings = [node_codebook[node_indexer.get_idx(basket[u])].bind(node_codebook[node_indexer.get_idx(basket[v])]) for u, v in edge_indexes]
# edge_bindings_tensor = torch.stack(edge_bindings)
# graph_hd_level1_multiset = torchhd.multiset(edge_bindings_tensor)



iteration = 1
max_iters = 50
should_break = False
graph_hv = graph_hd_level1.clone() * 0.5
while not should_break:
    scores = []
    for u, v in edge_to_check:
        if (u, v) not in found:
            edge_hv = node_codebook[u].bind(node_codebook[v])
            dot = torchhd.dot(graph_hv, edge_hv)
            if vsa == VSAModel.MAP:
                dot /= D
            scores.append(dot.item())
    idxs = list(range(len(scores)))
    sorted_indexec = sorted(idxs, key=lambda i: scores[i], reverse=True)
    for idx in sorted_indexec[::2]:
        u, v = edge_to_check[idx]
        if scores[idx] > 0.5:
            found.add((u, v))
            found.add((v, u))
            graph_hv -= node_codebook[u].bind(node_codebook[v])
            graph_hv -= node_codebook[v].bind(node_codebook[u])
            break
        elif scores[idx] <= 0:
            should_break = True
        else:
            continue
    iteration += 1
    if iteration > max_iters:
        break

    print(f"---- Iteration {iteration}\n")
    print(f"{scores=}")
    print(f"{sorted_indexec=}")
    print(f"{sorted(found)=}")
    print(f"{edge_to_check=}")


Counter({3: 11, 2: 10, 1: 5, 14: 2, 15: 2, 7: 1, 13: 1, 32: 1})
[(1, 2), (2, 1), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 2), (2, 3), (3, 2), (3, 14), (3, 32), (14, 3), (14, 15), (15, 14), (15, 3), (15, 3), (3, 15), (3, 13), (3, 3), (13, 3), (3, 3), (3, 2), (3, 3), (2, 3), (2, 3), (3, 2), (3, 3), (3, 2), (3, 3), (3, 1), (3, 15), (1, 3), (15, 3), (15, 3), (15, 3), (3, 15), (3, 2), (3, 3), (2, 3), (2, 2), (2, 2), (2, 3), (3, 2), (3, 1), (3, 2), (1, 3), (2, 3), (2, 3), (3, 3), (3, 2), (3, 1), (1, 3), (3, 15), (3, 1), (3, 2), (1, 3), (2, 3), (2, 3), (3, 3), (3, 7), (3, 14), (7, 3), (14, 3), (14, 3), (3, 15), (3, 14), (3, 32), (32, 3), (32, 3)]
Counter({(2, 2): 10, (2, 3): 9, (3, 2): 9, (3, 3): 8, (15, 3): 5, (3, 15): 5, (3, 1): 4, (1, 3): 4, (3, 14): 3, (14, 3): 3, (3, 32): 2, (32, 3): 2, (1, 2): 1, (2, 1): 1, (14, 15): 1, (15, 14): 1, (3, 13): 1, (13, 3): 1, (3, 7): 1, (7, 3): 1})
---- Iteration 2

scores=[0.7721274495124817, 1.8788331747055054, 9.96218490600586, -0.283

## Score

In [77]:
from src import evaluation_metrics


truth_counter = Counter([(node_indexer.get_idx(basket[u]), node_indexer.get_idx(basket[v])) for u, v in edge_indexes])
truth_counter_ones = {k: 1 for k in truth_counter}


found_counter = Counter(found)
print(f"{iteration=}")
print(f"{found_counter.total()=}")
p, _, f1 = evaluation_metrics.calculate_p_a_f1(pred=found_counter, true=truth_counter_ones)
print(f"{p=}")
print(f"{f1=}")


iteration=51
found_counter.total()=13
p=0.38461538461538464
f1=0.30303030303030304


## Explain Away Batched

In [78]:
def decode_order_one_counter(
        # self,
        embedding: torch.Tensor,                             # [B, D] or [D]
        unique_decode_nodes_batch: list[set[tuple[int,int]]],# length B
        max_iters: int = 50,
        threshold: float = 0.5,
        use_break: bool = True,
) -> list[Counter]:
    """
    Peel off top edges iteratively exactly as in your script,
    but do it for each graph in the batch.
    """

    # 1) ensure batch dimension
    if embedding.dim()==1:
        embedding = embedding.unsqueeze(0)
    B, D = embedding.shape

    # populate all codebooks/indexers
    # self._populate_nodes_codebooks()
    # self._populate_node_indexer()

    results: list[Counter] = []

    for b in range(B):
        # — build per‐graph data structures —
        graph_hv = embedding[b].clone()
        # which node‐indices are present?
        node_idxs = node_indexer.get_idxs(unique_decode_nodes_batch[b])
        # flatten+sort for reproducibility
        nodes_decoded_flat = sorted(node_idxs)

        # all possible directed pairs over those nodes
        edge_to_check = list(itertools.product(nodes_decoded_flat, nodes_decoded_flat))

        found = set()

        iteration = 1
        should_break = False
        while iteration <= max_iters and edge_to_check and not should_break:
            # 2) score every candidate
            scores = []
            for u,v in edge_to_check:
                eh = node_codebook[u].bind(node_codebook[v])
                s  = torchhd.dot(graph_hv, eh)
                # if MAP, normalize
                if vsa == VSAModel.MAP:
                    s = s / D
                scores.append(s.item())

            # 3) stop if even the top score is <= 0
            if max(scores) <= 0.0:
                break

            # 4) sort descending, pick every‐other slot
            idxs_sorted = sorted(range(len(scores)),
                                 key=lambda i: scores[i], reverse=True)


            for idx in idxs_sorted[::2]:
                u, v = edge_to_check[idx]
                if scores[idx] > threshold:
                    found.add((u, v))
                    found.add((v, u))
                    graph_hv -= node_codebook[u].bind(node_codebook[v])
                    graph_hv -= node_codebook[v].bind(node_codebook[u])
                    edge_to_check = [e for e in edge_to_check if e not in {(u, v), (v, u)}]
                    if use_break:
                        break
                elif scores[idx] <= 0:
                    should_break = True


            iteration += 1

        results.append(Counter(found))

    return results

In [83]:
# found_decode_order_one = decode_order_one_counter(
%timeit found_decode_order_one= decode_order_one_counter( embedding=graph_hd_level1.clone(), unique_decode_nodes_batch=[set(sorted(node_indexer.get_tuples(nodes_decoded_flatt)))], max_iters=50, threshold=0.5)
# _, _, f1 = evaluation_metrics.calculate_p_a_f1(pred=found_decode_order_one[0], true=truth_counter_ones)
# print(f1)

198 ms ± 2.74 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Explain Away (semi)Tensor

In [80]:
def decode_order_one_counter_fast(
        embedding: torch.Tensor,                             # [B, D] or [D]
        unique_decode_nodes_batch: list[set[tuple[int,int]]],# length B
        edge_codebook: torch.Tensor,                         # [N_edges, D]
        edge_indexer: TupleIndexer,
        node_indexer: TupleIndexer,
        vsa: VSAModel,
        D: int,
        max_iters: int = 50,
        threshold: float = 0.5,
) -> list[Counter]:
    # 1) batchify embeddings
    if embedding.dim()==1:
        embedding = embedding.unsqueeze(0)
    B, _ = embedding.shape

    results = []
    for b in range(B):
        graph_hv = embedding[b].clone()

        # 2) build the list of candidate edge *flat* indices once
        node_idxs = node_indexer.get_idxs(unique_decode_nodes_batch[b])
        pairs     = list(itertools.product(node_idxs, node_idxs))
        cand_idxs   = torch.tensor(edge_indexer.get_idxs(pairs), dtype=torch.long)

        found = set()
        iteration, should_break = 1, False

        while iteration <= max_iters and len(cand_idxs)>0 and not should_break:
            # 2) score all candidates at once
            slice_hvs = edge_codebook[cand_idxs]       # [E, D]
            sims       = slice_hvs.matmul(graph_hv)    # [E]
            if vsa == VSAModel.MAP:
                sims = sims / D

            # 3) break if top ≤ 0
            if sims.max() <= 0:
                break

            # 4) sort descending indices
            idxs_sorted = sims.argsort(descending=True)  # [E]

            # inner loop, every‐other slot
            for idx in idxs_sorted[::2].tolist():
                score = sims[idx].item()
                u,v   = edge_indexer.get_tuple(cand_idxs[idx].item())

                if score > threshold:
                    # exactly your original body:
                    found.add((u, v))
                    found.add((v, u))

                    graph_hv = graph_hv \
                               - node_codebook[u].bind(node_codebook[v]) \
                               - node_codebook[v].bind(node_codebook[u])

                    # remove both directions
                    to_remove = {
                        edge_indexer.get_idx((u, v)),
                        edge_indexer.get_idx((v, u))
                    }
                    mask = [(i.item() not in to_remove) for i in cand_idxs]
                    cand_idxs = cand_idxs[mask]
                    break

                elif score <= 0:
                    should_break = True
                    break
                # else continue searching

            iteration += 1

        results.append(Counter(found))

    return results

In [86]:
edge_codebook = cartesian_bind_tensor([node_codebook, node_codebook])
edge_indexer = TupleIndexer([28*6, 28*6])
# found_decode_order_one_fast = decode_order_one_counter_fast(
found_decode_order_one_fast = decode_order_one_counter_fast( embedding=graph_hd_level1.clone(), edge_codebook=edge_codebook, edge_indexer=edge_indexer, node_indexer=node_indexer, unique_decode_nodes_batch=[set(sorted(node_indexer.get_tuples(nodes_decoded_flatt)))], max_iters=50, threshold=0.5, vsa=vsa, D=D )
p, _, f1 = evaluation_metrics.calculate_p_a_f1(pred=found_decode_order_one_fast[0], true=truth_counter_ones)
print(p, f1)


0.6666666666666666 0.8
